In [130]:
import mysql.connector
import pandas as pd
from datetime import datetime
def get_data(table_name, report_date=None):
        # Parse and format the date
        if report_date:
            if isinstance(report_date, str):
                try:
                    # Try parsing from DD.MM.YYYY to YYYY-MM-DD
                    parsed_date = datetime.strptime(report_date, '%d.%m.%Y').strftime('%Y-%m-%d')
                except ValueError:
                    try:
                        # Already in correct format?
                        parsed_date = datetime.strptime(report_date, '%Y-%m-%d').strftime('%Y-%m-%d')
                    except ValueError:
                        raise ValueError("report_date must be in 'DD.MM.YYYY' or 'YYYY-MM-DD' format")
            else:
                parsed_date = report_date.strftime('%Y-%m-%d')

            query = f"SELECT * FROM {table_name} WHERE report_date < '{parsed_date}'"
        else:
            query = f"SELECT * FROM {table_name}"

        cursor.execute(query)

        # Fetch column names and data
        column_names = [desc[0] for desc in cursor.description]
        rows = cursor.fetchall()

        df = pd.DataFrame(rows, columns=column_names)
        return df

# Establish MySQL connection
conn = mysql.connector.connect(
    host="10.0.0.3",
    port = '3306',
    user="dashboard_report_vinamilk",
    password="6vtRhNxa79zPsfs5",
    database="dashboard_report_vinamilk"
)
cursor = conn.cursor()



df_tt = get_data("tiktok_template_5")
df_yt = get_data("googleads_performance_daily_raw_v1")
df_fb = get_data("facebook_performance_daily_raw")
# df_ga = get_data("ga4_performance_daily_raw")

if 'cursor' in locals():
    cursor.close()
if 'conn' in locals() and conn.is_connected():
    conn.close()
    print("MySQL connection closed.")

MySQL connection closed.


In [131]:
# Load mapping data
df_map_camp = pd.read_excel('/Users/khanhvx/Downloads/map.xlsx', sheet_name='campaign')
df_map_ad = pd.read_excel('/Users/khanhvx/Downloads/map.xlsx', sheet_name='ad2')
df_map_camp = df_map_camp.drop_duplicates(subset='campaign_id', keep='last')
df_map_ad = df_map_ad.drop_duplicates(subset='ad_id', keep='first')
# Clear existing name columns
df_fb = df_fb.drop(columns=['campaign_name', 'ad_name'])
# Merge campaign name
df_fb = df_fb.merge(df_map_camp[['campaign_id', 'campaign_name']], on='campaign_id', how='left')

# Merge ad (dd_name) from ad_id
df_fb = df_fb.merge(df_map_ad[['ad_id', 'ad_name']], on='ad_id', how='left')
df_fb.drop_duplicates(inplace= True)

# Clear existing name columns
df_yt = df_yt.drop(columns=['campaign', 'ad'])
# Merge campaign name
df_yt = df_yt.merge(df_map_camp[['campaign_id', 'campaign_name']], on='campaign_id', how='left')

# Merge ad (dd_name) from ad_id
df_yt = df_yt.merge(df_map_ad[['ad_id', 'ad_name']], on='ad_id', how='left')
df_yt.drop_duplicates(inplace= True)



In [132]:
import numpy as np
import pandas as pd # Make sure pandas is imported
from datetime import datetime

# ... (keep your rearrange_marker function and initial data cleaning) ...

campaign_name_expected_columns = ['camp_code', 'platform', 'camp_type', 'buying_type', 'free_text1', 'year', 'optimization',
                                  'buying_model', 'placement', 'freetext_2']

adgroup_name_expected_columns = ['noneed_1', 'noneed_2', 'noneed_3', 'noneed_4', 'noneed_5', 'noneed_6', 'noneed_7',
                                 'noneed_8', 'bidding_strategy', 'noneed_9']

# Original list with 9 names
ad_name_expected_columns = ['creative_type', 'creative_format', 'creative_length', 'noneed_9', 'noneed_10', 'noneed_11', 'noneed_12',
                            'noneed_13', 'freetext_3']

def split_and_assign(df, column, expected_columns):
    if column in df.columns:
        df[column] = df[column].fillna("").astype(str)
        split_cols = df[column].str.split('_', expand=True)

        num_expected = len(expected_columns)
        num_actual = split_cols.shape[1]

        # Determine how many columns to actually use and name (max is num_expected)
        cols_to_assign = min(num_actual, num_expected)

        # Take only the relevant columns from the split (up to num_expected)
        split_cols_final = split_cols.iloc[:, :cols_to_assign]

        # Assign the corresponding expected names
        split_cols_final.columns = expected_columns[:cols_to_assign]

        # Check for existing columns before joining to avoid duplicate column errors
        cols_to_drop_before_join = [col for col in split_cols_final.columns if col in df.columns]
        df = df.drop(columns=cols_to_drop_before_join, errors='ignore')

        df = df.join(split_cols_final)


    # ✅ Improved camp_code splitting (keep this logic as is)
    if "camp_code" in df.columns:
        # ... (your existing camp_code splitting logic) ...
        df["camp_code"] = df["camp_code"].fillna("").astype(str)
        # First, split from the right on last two hyphens → Start Date and End Date
        campaign_split = df["camp_code"].str.rsplit("-", n=2, expand=True)
        campaign_split_final = pd.DataFrame() # Initialize empty DataFrame

        if campaign_split.shape[1] == 3:
            prefix_part = campaign_split[0]
            start_date_str = campaign_split[1]
            end_date_str = campaign_split[2]

            # Now, split the prefix part into the first 4 components
            prefix_split = prefix_part.str.split("-", n=3, expand=True)
            # Ensure prefix_split has 4 columns, padding if necessary
            for i in range(prefix_split.shape[1], 4):
                 prefix_split[i] = "" # Add empty columns if split is short
            prefix_split.columns = ["Funnel", "Category", "Brand Name", "Campaign Name"]


            # Combine all parts
            campaign_split_final = pd.concat([prefix_split, start_date_str, end_date_str], axis=1)
            campaign_split_final.columns = ["Funnel", "Category", "Brand Name", "Campaign Name", "Start Date", "End Date"]

             # Drop existing columns before joining to avoid conflict
            df = df.drop(columns=[col for col in campaign_split_final.columns if col in df.columns], errors="ignore")
            df = df.join(campaign_split_final)
        # else: handle cases where camp_code doesn't split into 3 parts if needed


    # ✅ free_text1 splitting (keep this logic as is)
    if "free_text1" in df.columns:
        # ... (your existing free_text1 splitting logic) ...
        df["free_text1"] = df["free_text1"].fillna("").astype(str)
        freetext_split = df["free_text1"].str.split("-", expand=True)

        max_cols = 6
        # Select up to max_cols, don't error if fewer exist
        freetext_split = freetext_split.iloc[:, :min(freetext_split.shape[1], max_cols)]

        # Pad with NaN if fewer than max_cols resulted from the split
        for i in range(freetext_split.shape[1], max_cols):
            freetext_split[i] = np.nan # Use standard column names

        # Assign standard names
        freetext_split.columns = ["Audience", "Gender", "Age", "Region", "Device", "Os"][:freetext_split.shape[1]] # Only assign names for existing columns

        df = df.drop(columns=[col for col in freetext_split.columns if col in df.columns], errors="ignore")
        df = df.join(freetext_split)


    # ✅ Target Audience creation (keep this logic as is)
    if "Gender" in df.columns and "Age" in df.columns:
         # Ensure columns exist before proceeding
        df["Gender"] = df["Gender"].fillna("NA").astype(str) # Handle potential NaNs from split
        df["Age"] = df["Age"].fillna("0000").astype(str).str.zfill(4) # Handle NaNs and ensure padding
        # Use .str accessor safely
        df["TA"] = df["Gender"] + df["Age"].str[:2] + "-" + df["Age"].str[2:]


    # ✅ Date formatting
    # Ensure 'year' column exists and handle potential errors before using iloc[0]
    current_year = datetime.now().year # Default year if 'year' column is missing/empty
    if "year" in df.columns and not df["year"].isnull().all():
        # Try converting the first valid year value to int
        try:
            first_valid_year_index = df['year'].first_valid_index()
            if first_valid_year_index is not None:
                current_year = int(df['year'].loc[first_valid_year_index])
            else:
                 print("Warning: 'year' column has no valid values. Using current system year.")
        except (ValueError, TypeError):
            print(f"Warning: Could not convert value in 'year' column to integer. Using current system year: {current_year}")


    if "Start Date" in df.columns and "End Date" in df.columns:
        def parse_ddmm(val, year_to_use):
            if pd.isna(val) or val == '':
                return pd.NaT
            try:
                # Handle potential float inputs from splits
                val_str = str(val).split('.')[0] if isinstance(val, float) else str(val)
                return datetime.strptime(val_str, "%d%m").replace(year=year_to_use)
            except (ValueError, TypeError):
                 # Handle cases like 'NaT', '', invalid formats
                return pd.NaT

        # Apply using the determined year
        df["Start Date"] = df["Start Date"].apply(lambda x: parse_ddmm(x, current_year))
        df["End Date"] = df["End Date"].apply(lambda x: parse_ddmm(x, current_year))


    # ✅ Combine creative fields (keep this logic as is)
    if all(col in df.columns for col in ["creative_type", "creative_format", "creative_length"]):
        df["ad_format"] = (df["creative_type"].fillna("").astype(str) + "_" +
                           df["creative_format"].fillna("").astype(str) + "_" +
                           df["creative_length"].fillna("").astype(str))


    # ✅ Drop unnecessary columns (keep this logic as is)
    df = df.drop(columns=[col for col in df.columns if "noneed" in col], errors="ignore")

    return df


# ... (keep your rename and process_campaign_data function) ...

def process_campaign_data(df):
    """Processes a DataFrame by filtering campaigns, splitting columns, and formatting TA & dates."""

    # Ensure 'campaign_name' exists before filtering
    if 'campaign_name' not in df.columns:
        print("Warning: 'campaign_name' column not found. Skipping filtering.")
        df_filtered = df.copy() # Process the whole df if no campaign_name
    else:
         # Filter DataFrame based on campaign prefix (handle NaN safely)
        df_filtered = df[df['campaign_name'].fillna('').str.startswith(('AW'))].copy()


    # Apply splitting logic - rename 'ad_group' -> 'ad_group_name' if necessary here or before
    # Make sure the column names passed match exactly what's in df_filtered
    df_filtered = split_and_assign(df_filtered, 'campaign_name', campaign_name_expected_columns)
    # *** Check if 'ad_group' or 'ad_group_name' exists after renames ***
    if 'ad_group_name' in df_filtered.columns:
         df_filtered = split_and_assign(df_filtered, 'ad_group_name', adgroup_name_expected_columns)
    elif 'ad_group' in df_filtered.columns:
         print("Warning: Processing 'ad_group' instead of 'ad_group_name'")
         df_filtered = split_and_assign(df_filtered, 'ad_group', adgroup_name_expected_columns)

    # *** Check if 'ad_name' exists after renames ***
    if 'ad_name' in df_filtered.columns:
        df_filtered = split_and_assign(df_filtered, 'ad_name', ad_name_expected_columns)

    return df_filtered

# Assuming df_tt, df_fb, df_yt are loaded DataFrames
# Perform renames *before* calling process_campaign_data
df_yt = df_yt.rename(columns={'campaign': 'campaign_name', 'ad': 'ad_name', 'ad_group': 'ad_group_name'})
# Add similar renames for df_fb and df_tt if they use different original names

df_yt_filter = process_campaign_data(df_yt)
df_fb_filter = process_campaign_data(df_fb) # Make sure df_fb has 'campaign_name', etc.
df_tt_filter = process_campaign_data(df_tt) # Make sure df_tt has 'campaign_name', etc.

In [133]:
# Drop the old 'platform' column if it exists
df_yt_filter = df_yt_filter.drop(columns=['platform'], errors='ignore')

# Add new 'Platform' column with value 'Youtube'
df_yt_filter['Platform'] = 'YouTube'

# Facebook
df_fb_filter = df_fb_filter.drop(columns=['platform'], errors='ignore')
df_fb_filter['Platform'] = 'Facebook'

# TikTok
df_tt_filter = df_tt_filter.drop(columns=['platform'], errors='ignore')
df_tt_filter['Platform'] = 'TikTok'

In [134]:
df_yt_filter['Cost'] = np.where(
    df_yt_filter['currency_code'] == 'USD',
    25100 * df_yt_filter['cost'],
    1 * df_yt_filter['cost']
)

In [135]:
import pandas as pd

def process_dataframe(df, column_mapping, numeric_cols, extra_processing=None, column_order=None):
    """General function to process dataframes."""
    # Select and rename columns
    existing_columns = [col for col in column_mapping if col in df.columns]
    df = df[existing_columns].rename(columns=column_mapping).copy()
    
    # Convert 'Date' column
    df['report_date'] = pd.to_datetime(df['report_date'], errors='coerce')
    df['report_date'] = pd.to_datetime(df['report_date'], errors='coerce').dt.date

    # Convert numeric columns
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Apply dataset-specific processing
    if extra_processing:
        extra_processing(df)
    
    # Ensure all expected columns exist
    if column_order:
        for col in column_order:
            if col not in df.columns:
                df[col] = None
        df = df[column_order]
    
    return df


# Define column mappings & numeric columns for each dataset
youtube_mapping = {'report_date': 'report_date',
    'Category': 'Category',
    'Funnel': 'Funnel',
    'platform': 'Platform',
    'Brand Name': 'Brand',
    'placement': 'Video_Range',
    'creative_type': 'Creative_Type',
    'creative_format': 'Creative_Format',
    'creative_length': 'Creative_Length',
    'buying_model': 'Buying_Method',
    'TA': 'TA',
    'Region': 'Region',
    'campaign_name': 'Campaign_Name',
    'camp_code': 'Campaign_code',
    'Campaign Name': 'Campaign',
    'Start Date': 'Start_Date',
    'End Date': 'End_Date',
    'freetext_2': 'freetext_2',
    'freetext_3': 'freetext_3',
    'camp_type': 'Format',
    'impressions': 'Impression',
    'clicks': 'Clicks',
    'views': 'Views',
    'currency_code': 'Currency_Code',
    'cost': 'Cost',
    'engagements': 'Engagements',
    'video_25': 'Video_Plays_25',
    'video_50': 'Video_Plays_50',
    'video_75': 'Video_Plays_75',
    'video_100': 'Video_Plays_100'}  
tiktok_mapping = {'report_date': 'report_date',
    'Category': 'Category',
    'Funnel': 'Funnel',
    'platform': 'Platform',
    'Brand Name': 'Brand',
    'TA': 'TA',
    'Region': 'Region',
    'camp_type': 'Format',
    'creative_type': 'Creative_Type',
    'creative_format': 'Creative_Format',
    'creative_length': 'Creative_Length',
    'buying_model': 'Buying_Method',
    'campaign_name': 'Campaign_Name',
    'camp_code': 'Campaign_code',
    'Campaign Name': 'Campaign',
    'Start Date': 'Start_Date',
    'End Date': 'End_Date',
    'freetext_2': 'freetext_2',
    'freetext_3': 'freetext_3',
    'spend': 'Cost',
    'impressions': 'Impression',
    'clicks_destination': 'Clicks',
    'engagements': 'Engagements',
    'profile_visits': 'profile_visits',
    '2_second_video_views': '23s_Video_Views',
    '6_second_video_views': 'Views',
    'engaged_view': '6s_Engaged_Views',
    'video_views_p25': 'Video_Plays_25',
    'video_views_p50': 'Video_Plays_50',
    'video_views_p75': 'Video_Plays_75',
    'video_views_p100': 'Video_Plays_100'}
facebook_mapping = {'report_date': 'report_date',
    'Category': 'Category',
    'Funnel': 'Funnel',
    'platform': 'Platform',
    'Brand Name': 'Brand',
    'camp_type': 'Format',
    'creative_type': 'Creative_Type',
    'creative_format': 'Creative_Format',
    'creative_length': 'Creative_Length',
    'buying_model': 'Buying_Method',
    'TA': 'TA',
    'Region': 'Region',
    'campaign_name': 'Campaign_Name',
    'camp_code': 'Campaign_code',
    'Campaign Name': 'Campaign',
    'Start Date': 'Start_Date',
    'End Date': 'End_Date',
    'freetext_2': 'freetext_2',
    'freetext_3': 'freetext_3',
    'impression': 'Impression',
    'cost': 'Cost',
    'post_engagement': 'Engagements',
    'video_played_3': '23s_Video_Views',
    'thruplays': 'Views',
    'video_played_25': 'Video_Plays_25',
    'video_played_50': 'Video_Plays_50',
    'video_played_75': 'Video_Plays_75',
    'video_played_complete': 'Video_Plays_100',
    'link_click': 'Clicks'}

youtube_numeric_cols = ['Impression', 'Clicks', 'Views', 'Cost', 'Engagements', 
    'Video_Played_25', 'Video_Played_50', 'Video_Played_75', 
    'Video_Played_100', 'Conversions']
tiktok_numeric_cols = ['Cost', 'Impression', 'Clicks', 'Engagement',
    '2s_Video_Views', '6s_Video_Views', '6s_Engaged_Views',
   'Video_Played_25', 'Video_Played_50', 'Video_Played_75', 
    'Video_Played_100']
facebook_numeric_cols = ['Engagement', '3s_Video_Plays', 'Impression',
    'Cost', 'Clicks', 'Video_Played_25', 'Video_Played_50', 'Video_Played_75', 
    'Video_Played_100', 'Thruplays']


# Process datasets
report_campaign_overall_youtube = process_dataframe(df_yt_filter, youtube_mapping, youtube_numeric_cols)
report_campaign_overall_tiktok = process_dataframe(df_tt_filter, tiktok_mapping, tiktok_numeric_cols)
report_campaign_overall_facebook = process_dataframe(df_fb_filter, facebook_mapping, facebook_numeric_cols)

# Display first few rows
report_campaign_overall_youtube.head()
report_campaign_overall_tiktok.head()
report_campaign_overall_facebook.head()

report_campaign_overall_youtube['Platform'] = 'YouTube'
report_campaign_overall_tiktok['Platform'] = 'TikTok'
report_campaign_overall_facebook['Platform'] = 'Facebook'

report_campaign_overall_total = pd.concat([report_campaign_overall_youtube, report_campaign_overall_tiktok, report_campaign_overall_facebook], ignore_index=True)

# ✅ Step 4: Summarize at higher level (Brand + Campaign)
summary_columns = ['report_date', 'Category', 'Platform', 'Brand', 'Campaign', 'Format', 'Buying_Method', 'TA', 'Region', 'Start_Date', 'End_Date', 'Campaign_code', 'Funnel','freetext_3', 'freetext_2', 'Creative_Type', 'Creative_Length', 'Creative_Format']
metric_columns = ['Impression', 'Clicks', 'Views', 'Cost', 'Engagements',
                  'Video_Plays_25', 'Video_Plays_50', 'Video_Plays_75', 'Video_Plays_100']

# Filter only metrics that exist in the dataset
existing_metrics = [col for col in metric_columns if col in report_campaign_overall_total.columns]

# Group and aggregate
report_campaign_summary = (
    report_campaign_overall_total
    .groupby(summary_columns)[existing_metrics]
    .sum()
    .reset_index()
)

report_campaign_summary

,report_date,Category,Platform,Brand,Campaign,Format,Buying_Method,TA,Region,Start_Date,...,Creative_Format,Impression,Clicks,Views,Cost,Engagements,Video_Plays_25,Video_Plays_50,Video_Plays_75,Video_Plays_100
0,2024-12-29,SBNL,TikTok,SurePreventGold,SBGPromo,REA,CPM,F25-44,20C,2025-04-21,...,VER,0,0,0,0.000000,0,0.00,0.000,0.000,0.000
1,2024-12-29,SBNL,TikTok,SurePreventGold,SBGPromo,REA,CPM,F25-44,20C,2025-04-21,...,VER,0,0,0,0.000000,0,0.00,0.000,0.000,0.000
2,2024-12-29,SBNL,TikTok,SurePreventGold,SBGPromo,REA,CPM,F25-44,20C,2025-04-21,...,VER,0,0,0,0.000000,0,0.00,0.000,0.000,0.000
3,2024-12-29,SBNL,TikTok,SurePreventGold,SBGPromo,REA,CPM,F25-44,20C,2025-04-21,...,VER,0,0,0,0.000000,0,0.00,0.000,0.000,0.000
4,2024-12-29,SBNL,TikTok,SurePreventGold,SBGPromo,REA,CPM,F25-44,20C,2025-04-21,...,VER,0,0,0,0.000000,0,0.00,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26781,2025-05-06,SN,YouTube,GreenFarm,GreenFarmCorePromotionMar-Apr,VRC,CPM,F25-44,35C,2025-03-27,...,VER,377212,979,9339,197.855589,19621,28032.89,12023.730,7062.984,4945.542
26782,2025-05-06,SN,YouTube,GreenFarm,GreenFarmCorePromotionMar-Apr,VRC,CPM,F25-44,35C,2025-03-27,...,HOR,103451,77,0,208.400592,0,101752.92,99971.660,99098.730,98450.100
26783,2025-05-06,SN,YouTube,GreenFarm,GreenFarmCorePromotionMar-Apr,VRC,CPM,F25-44,35C,2025-03-27,...,VER,25234,53,568,37.771612,686,13933.51,11899.667,11265.742,10907.211
26784,2025-05-06,SN,YouTube,GreenFarm,GreenFarmCorePromotionMar-Apr,VRC,CPM,F25-44,35C,2025-03-27,...,HOR,100199,87,0,203.813672,0,98507.10,96700.200,95780.800,95162.500


In [136]:
report_campaign_summary['Content_Angle'] = report_campaign_summary['freetext_3'].str.split('-').str[0]

In [79]:
import pandas as pd
import numpy as np 
# Assuming df_ga is your DataFrame loaded previously

# --- 1. Split sessionManualCampaignName ---
print("Processing session_manual_campaign_name...")
target_cols_1 = ['Campaign', 'Category', 'Brand', 'Dentsu', 'Start_Date']
n_target_1 = len(target_cols_1)

# Perform the split safely, filling NaNs in the source column first
split_data_1 = df_ga['session_manual_campaign_name'].fillna('').astype(str).str.split('_', expand=True)
n_actual_1 = split_data_1.shape[1]
print(f"  Split produced {n_actual_1} columns (expected {n_target_1}).")

# Select the minimum of actual columns produced and target columns
cols_to_assign_1 = min(n_target_1, n_actual_1)

# Assign the data for the columns that exist in both
df_ga[target_cols_1[:cols_to_assign_1]] = split_data_1.iloc[:, :cols_to_assign_1]

# If the split produced fewer columns than expected, fill the remaining target columns with NaN/None
if n_actual_1 < n_target_1:
    print(f"  Padding {n_target_1 - n_actual_1} missing columns.")
    for i in range(n_actual_1, n_target_1):
        df_ga[target_cols_1[i]] = pd.NA # Or np.nan or None or ''


# --- 2. Split sessionManualAdContent ---
print("Processing session_manual_ad_content...")
target_cols_2 = ['Content_Angle', 'Tên Content', 'Tên Thiết Kế', 'Creative Format']
n_target_2 = len(target_cols_2)

# Perform the split safely
split_data_2 = df_ga['session_manual_ad_content'].fillna('').astype(str).str.split('_', expand=True)
n_actual_2 = split_data_2.shape[1]
print(f"  Split produced {n_actual_2} columns (expected {n_target_2}).")

cols_to_assign_2 = min(n_target_2, n_actual_2)
df_ga[target_cols_2[:cols_to_assign_2]] = split_data_2.iloc[:, :cols_to_assign_2]

if n_actual_2 < n_target_2:
    print(f"  Padding {n_target_2 - n_actual_2} missing columns.")
    for i in range(n_actual_2, n_target_2):
        df_ga[target_cols_2[i]] = pd.NA


# --- 3. Split sessionManualMedium ---
print("Processing session_manual_medium...") # Corrected column name based on code
target_cols_3 = ['Buying Method', 'Bid Strategy/Objective', 'Targeting Method', 'Audience', 'Age', 'Gender', 'Region']
n_target_3 = len(target_cols_3)

# Check if 'session_manual_mmedium' exists, otherwise use 'sessionManualMedium' or correct it
source_col_3 = 'session_manual_medium' #<-- Make sure this is the EXACT column name in df_ga
if source_col_3 not in df_ga.columns:
     # Attempt common variations or raise error
     if 'sessionManualMedium' in df_ga.columns:
         source_col_3 = 'sessionManualMedium'
     elif 'session_manual_mmedium' in df_ga.columns: # Check the original typo
         source_col_3 = 'session_manual_mmedium'
     else:
         raise KeyError(f"Column for medium split not found. Tried 'session_manual_medium', 'sessionManualMedium', 'session_manual_mmedium'. Please verify column name.")
print(f"  Using source column: {source_col_3}")

# Perform the split safely
split_data_3 = df_ga[source_col_3].fillna('').astype(str).str.split('_', expand=True)
n_actual_3 = split_data_3.shape[1]
print(f"  Split produced {n_actual_3} columns (expected {n_target_3}).")

cols_to_assign_3 = min(n_target_3, n_actual_3)
df_ga[target_cols_3[:cols_to_assign_3]] = split_data_3.iloc[:, :cols_to_assign_3]

if n_actual_3 < n_target_3:
    print(f"  Padding {n_target_3 - n_actual_3} missing columns.")
    for i in range(n_actual_3, n_target_3):
        df_ga[target_cols_3[i]] = pd.NA


# --- 4. Split sessionManualSource (Platform_Objective-BuyingType) ---
print("Processing session_manual_source...")
# First split by '_'
split_source_1 = df_ga['session_manual_source'].fillna('_').astype(str).str.split('_', n=1, expand=True) # n=1 ensures max 2 columns

# Assign temporary columns, handling cases where split might not occur
df_ga['Platform_temp'] = split_source_1[0]
if split_source_1.shape[1] > 1:
    df_ga['Objective-BuyingType_temp'] = split_source_1[1]
else:
    df_ga['Objective-BuyingType_temp'] = '' # Or pd.NA

# Then split the second part by '-'
split_source_2 = df_ga['Objective-BuyingType_temp'].fillna('-').astype(str).str.split('-', n=1, expand=True) # n=1 ensures max 2 columns

# Assign final columns
df_ga['Platform'] = df_ga['Platform_temp'] # Assign directly from first split
df_ga['Objective'] = split_source_2[0]
if split_source_2.shape[1] > 1:
    df_ga['Buying Type'] = split_source_2[1]
else:
    df_ga['Buying Type'] = '' # Or pd.NA

# Drop intermediate columns
df_ga.drop(columns=['Platform_temp', 'Objective-BuyingType_temp'], inplace=True)


Processing session_manual_campaign_name...
  Split produced 10 columns (expected 5).
Processing session_manual_ad_content...
  Split produced 5 columns (expected 4).
Processing session_manual_medium...
  Using source column: session_manual_medium
  Split produced 7 columns (expected 7).
Processing session_manual_source...


In [80]:
df_lookup = pd.read_excel('/Users/khanhvx/Downloads/dimensions.xlsx')
df_lookup['code'] = df_lookup['code'].astype(str)

# Create the mapping dictionary from df_lookup
decode_map = dict(zip(df_lookup['code'], df_lookup['name']))

# List of columns to decode
cols_to_decode = [
    'Campaign', 'Category', 'Brand', 'Dentsu', 'Start_Date',
    'Content Angle', 'Tên Content', 'Tên Thiết Kế', 'Creative Format',
    'Buying Method', 'Bid Strategy/Objective', 'Targeting Method',
    'Audience', 'Age', 'Gender', 'Region',
    'Platform', 'Objective-BuyingType'
]

# 4. Decode in-place by mapping string version of the value
for col in cols_to_decode:
    if col in df_ga.columns:
        df_ga[col] = df_ga[col].astype(str).map(decode_map).fillna(df_ga[col])

df_ga

,report_date,session_manual_campaign_name,session_manual_ad_content,session_manual_medium,session_manual_source,sessions,engaged_sessions,add_to_carts,purchase_revenue,purchaser_rate,...,Buying Method,Bid Strategy/Objective,Targeting Method,Audience,Age,Gender,Region,Platform,Objective,Buying Type
0,2025-04-11,3166_sbte_optimumgold_3011_2025/03/05,323_323_323_32,86_85_52_60_268_359_177,184_362,2110,2056,0,0.0,0.0,...,CPM,Impression,Affinity,none,1844,Female,Toàn Quốc,Youtube,362,None
1,2025-04-11,3224_sn_greenfarm_3011_2025/03/28,323_323_323_na,86_85_71_60_1545_359_177,184_362,963,942,3,0.0,0.0,...,CPM,Impression,Broad,none,2544,Female,Toàn Quốc,Youtube,362,None
2,2025-04-11,3166_sbte_optimumgold_3011_2025/03/05,323_323_323_68,86_85_52_60_268_359_177,8_197,920,901,1,0.0,0.0,...,CPM,Impression,Affinity,none,1844,Female,Toàn Quốc,Facebook,197,None
3,2025-04-11,3206_sbte_yokogold_3011_2025/03/25,323_323_323_50,86_85_52_60_268_359_177,8_125,604,599,4,0.0,0.0,...,CPM,Impression,Affinity,none,1844,Female,Toàn Quốc,Facebook,125,None
4,2025-04-11,3166_sbte_optimumgold_3011_2025/03/05,323_323_323_68,86_85_52_60_268_359_177,184_362,567,545,0,0.0,0.0,...,CPM,Impression,Affinity,none,1844,Female,Toàn Quốc,Youtube,362,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2848,2025-04-27,3216_sn_greenfarm_3011_2025/03/26,323_323_323_226,86_30_71_60_25_359_177,8_669,1,0,0,0.0,0.0,...,CPM,Reach,Broad,none,18-24,Female,Toàn Quốc,Facebook,669,None
2849,2025-04-27,3292_sbnl_surepreventgold_3011_2025/04/21,3294_3259_1113_226,86_85_201_60_155_361_2630,8_197,1,1,0,0.0,0.0,...,CPM,Impression,Prospecting,none,45-55,People,TOP20C,Facebook,197,None
2850,2025-04-27,3324_sn_greenfarm_3011_2025/04/22,3319_323_323_226,185_6_71_60_25_359_177,76_367,1,0,0,0.0,0.0,...,CPV,View,Broad,none,18-24,Female,Toàn Quốc,Tiktok,367,None
2851,2025-04-27,3335_sn_greenfarm_3011_2025/04/24,3337_323_323_22,70_108_71_325_81_361_na,95_183,1,1,0,0.0,0.0,...,Reservation,NONE,Broad,Website ALL,BROAD,People,na,PR,183,None


In [85]:
df_ga= pd.read_excel('/Users/khanhvx/Downloads/df_ga.xlsx')

# import pandas as pd

# # Copy your original DataFrame

# 1. Safe split for sessionManualCampaignName (expecting 5 parts)
# df_ga[['Campaign Name', 'Cate', 'Brand', 'Dentsu', 'Start Date']] = (
#     df_ga['session_manual_campaign_name']
#     .str.extract(r'([^_]+)_([^_]+)_([^_]+)_([^_]+)_([^_]+)')
# )

# # 2. Safe split for sessionManualAdContent (expecting 4 parts)
# df_ga[['Content_Angle', 'Tên Content', 'Tên Thiết Kế', 'Creative Format']] = (
#     df_ga['session_manual_ad_content']
#     .str.extract(r'([^_]+)_([^_]+)_([^_]+)_([^_]+)')
# )

# # 3. Safe split for sessionManualMedium (expecting 7 parts)
# df_ga[['Buying Method', 'Bid Strategy/Objective', 'Targeting Method', 'Audience', 'Age', 'Gender', 'Geo']] = (
#     df_ga['session_manual_medium']
#     .str.extract(r'([^_]+)_([^_]+)_([^_]+)_([^_]+)_([^_]+)_([^_]+)_([^_]+)')
# )

# # 4. Safe split for sessionManualSource (Platform_Objective-BuyingType)
# # First split by '_'
# df_ga[['Platform', 'Objective-BuyingType']] = (
#     df_ga['session_manual_source']
#     .str.extract(r'([^_]+)_([^_]+)')
# )


# Step 1: Replace 'Toàn Quốc' with 'NAT' in Geo
df_ga['Geo'] = df_ga['Geo'].replace('Toàn Quốc', 'NAT')

# Step 2: Create TA column from Gender and Age
def make_ta(gender, age):
    try:
        first_letter = gender[0].upper()
        age_range = f"{age[:2]}-{age[2:]}"
        return f"{first_letter}{age_range}"
    except:
        return None  # or you can return f"{gender}_{age}" as fallback

df_ga['TA'] = df_ga.apply(lambda row: make_ta(row['Gender'], row['Age']), axis=1)
df_ga

,sessions,engaged_sessions,add_to_carts,purchase_revenue,ecommerce_purchases,screen_page_views,Campaign Name,Cate,Brand,Start Date,Content_Angle,Creative Format,Buying Method,Age,Gender,Geo,Platform,report_date,TA
0,4446,4390,0,0.0,0,24,GiamV30K,cpas,nc,optimumgold,107,Photo-Multi,CPAS,BROAD,People,NAT,Facebook,2025-03-16,PBR-OAD
1,3572,3553,0,0.0,0,8,GiamV30K,cpas,nc,optimumgold,107,Photo-Multi,CPAS,BROAD,People,NAT,Facebook,2025-03-20,PBR-OAD
2,3465,3428,0,0.0,0,16,GiamV30K,cpas,nc,optimumgold,107,Photo-Multi,CPAS,BROAD,People,NAT,Facebook,2025-03-17,PBR-OAD
3,3299,3190,9,0.0,0,265,SCAThematic2025,sca,scavinamilk,2025/03/03,ITVC,Video15s,CPM,1844,People,NAT,Youtube,2025-03-03,P18-44
4,3177,3078,6,0.0,0,75,PackageChange,sbte,optimumgold,2025/03/05,323,Video6s,CPM,1844,Female,NAT,Youtube,2025-03-30,F18-44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3658,5,5,0,0.0,0,0,Promotion,sn,fm,2025/01/03,CTKD,Short 6s,CPM,2544,Female,NAT,Youtube,2025-03-06,F25-44
3659,5,4,0,0.0,0,0,Promotion,sn,fm,2025/01/03,CTKD,Short 6s,CPM,2544,Female,NAT,Youtube,2025-03-07,F25-44
3660,5,4,0,0.0,0,0,Promotion,sn,fm,2025/01/03,CTKD,Video6s,CPM,2544,Female,NAT,Youtube,2025-03-03,F25-44
3661,5,4,0,0.0,0,0,Promotion,sn,fm,2025/01/03,CTKD,Video15s,CPM,2544,Female,NAT,Youtube,2025-03-07,F25-44


In [86]:
df_ga_summary = df_ga.groupby([
    'report_date', 
    'Campaign Name', 
    'Cate', 
    'Brand', 
    'Start Date', 
    'Buying Method', 
    'Geo', 
    'Platform', 
    'TA',
    'Content_Angle'
])[[
    'sessions', 
    'engaged_sessions', 
    'add_to_carts', 
    'purchase_revenue', 
    'ecommerce_purchases', 
    'screen_page_views'
]].sum().reset_index()

df_ga_summary = df_ga_summary.rename(columns={
    'Campaign Name': 'Campaign',
    'Cate': 'Category',
    'Start Date': 'Start_Date',
    'Buying Method': 'Buying_Method',
    'Geo': 'Region'
})
df_ga_summary


,report_date,Campaign,Category,Brand,Start_Date,Buying_Method,Region,Platform,TA,Content_Angle,sessions,engaged_sessions,add_to_carts,purchase_revenue,ecommerce_purchases,screen_page_views
0,2025-03-01,225,sca,vinamilk,2025/02/27,CPM,2598-2599,Tiktok,F25-44,323,230,227,0,0.000000e+00,0,0
1,2025-03-01,225,sca,vinamilk,2025/02/27,CPM,2598-2599,Tiktok,P18--24,323,301,299,0,0.000000e+00,0,6
2,2025-03-01,225,sca,vinamilk,2025/02/27,CPM,2599-2598,Youtube,F25-44,323,946,896,0,0.000000e+00,0,15
3,2025-03-01,D2CPMAX,khác,none,2025/02/04,CPC,NAT,Google,P18-44,pmax,22,19,3,1.221789e+06,6,86
4,2025-03-01,GFSustain,sn,greenfarm,2025/02/22,CPM,TOP33C,Facebook,F18--24,323,68,61,5,0.000000e+00,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2803,2025-05-01,SocialAWO,sbte,optimum,2025/03/10,CPV,NAT,Facebook,F20-40,3286,94,94,0,0.000000e+00,0,58
2804,2025-05-01,SocialAWO,sbte,optimum,2025/03/19,CPV,NAT,Tiktok,F20-40,3322,31,28,0,0.000000e+00,0,30
2805,2025-05-01,SocialAWO,sca,scavinamilk,2025/04/11,CPM,2598-2599,Facebook,F18-44,323,13,7,1,0.000000e+00,0,1
2806,2025-05-01,SocialAWO,sca,scavinamilk,2025/04/28,CPM,TOP37C,Facebook,F18-44,3345,5,4,1,0.000000e+00,0,0


In [67]:
columns_to_drop = [
    'session_manual_campaign_name', 'session_manual_ad_content',
    'session_manual_medium', 'session_manual_source', 'Dentsu',
    'Tên Content', 'Tên Thiết Kế', 'Bid Strategy/Objective',
    'Objective', 'Buying Type', 'Targeting Method', 'Region', 'Gender', 'Age'
]

df_ga = df_ga.drop(columns=columns_to_drop)

KeyError: "['session_manual_campaign_name', 'session_manual_ad_content', 'session_manual_medium', 'session_manual_source', 'Dentsu', 'Tên Content', 'Tên Thiết Kế', 'Bid Strategy/Objective', 'Objective', 'Buying Type', 'Targeting Method', 'Region'] not found in axis"

In [137]:
import pandas as pd

# Assuming df_ga_summary and report_campaign_summary are already loaded

merge_cols = ['Campaign', 'report_date', 'Category', 'Region', 'Platform', 'Brand', 'TA', 'Buying_Method', 'Content_Angle']

# --- Refined Normalization ---
def normalize_cols(df, cols, prefix):
    for col in cols:
        # Handle dates explicitly first if they are datetime objects
        if pd.api.types.is_datetime64_any_dtype(df[col]):
             # Normalize to 'YYYY-MM-DD' format. Adjust if needed.
            df[f'{prefix}_{col}'] = pd.to_datetime(df[col]).dt.strftime('%Y-%m-%d')
        else:
            # Apply string normalization for others
            df[f'{prefix}_{col}'] = df[col].astype(str).str.strip().str.lower().str.replace(r'\s+', '', regex=True)
            # Handle potential 'nan' strings if they should be treated as actual NaN for merging (optional)
            # df[f'{prefix}_{col}'] = df[f'{prefix}_{col}'].replace('nan', pd.NA)
    return df

# Apply normalization
df_ga_summary = normalize_cols(df_ga_summary.copy(), merge_cols, 'ga') # Use .copy() to avoid SettingWithCopyWarning
report_campaign_summary = normalize_cols(report_campaign_summary.copy(), merge_cols, 'total')

# Step 2: Merge using the helper columns
merged_df = pd.merge(
    report_campaign_summary,
    df_ga_summary,
    how='left', # TRY 'left' or 'outer' HERE FOR DEBUGGING
    right_on=[f'ga_{col}' for col in merge_cols],
    left_on=[f'total_{col}' for col in merge_cols],
    suffixes=('', '_total')
)


# --- End Inspection ---


# Step 3: Drop helper columns (only if merge looks okay)
if not merged_df.empty:
     merged_df = merged_df.drop(columns=[f'ga_{col}' for col in merge_cols] + [f'total_{col}' for col in merge_cols] + [f'{col}_total' for col in merge_cols])

merged_df

,report_date,Category,Platform,Brand,Campaign,Format,Buying_Method,TA,Region,Start_Date,...,Video_Plays_75,Video_Plays_100,Content_Angle,Start_Date_total,sessions,engaged_sessions,add_to_carts,purchase_revenue,ecommerce_purchases,screen_page_views
0,2024-12-29,SBNL,TikTok,SurePreventGold,SBGPromo,REA,CPM,F25-44,20C,2025-04-21,...,0.000,0.000,HTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-12-29,SBNL,TikTok,SurePreventGold,SBGPromo,REA,CPM,F25-44,20C,2025-04-21,...,0.000,0.000,CTKD chung,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-12-29,SBNL,TikTok,SurePreventGold,SBGPromo,REA,CPM,F25-44,20C,2025-04-21,...,0.000,0.000,MayHutBui,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-12-29,SBNL,TikTok,SurePreventGold,SBGPromo,REA,CPM,F25-44,20C,2025-04-21,...,0.000,0.000,Thematic,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-12-29,SBNL,TikTok,SurePreventGold,SBGPromo,REA,CPM,F25-44,20C,2025-04-21,...,0.000,0.000,Thematic,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26781,2025-05-06,SN,YouTube,GreenFarm,GreenFarmCorePromotionMar-Apr,VRC,CPM,F25-44,35C,2025-03-27,...,7062.984,4945.542,NORTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26782,2025-05-06,SN,YouTube,GreenFarm,GreenFarmCorePromotionMar-Apr,VRC,CPM,F25-44,35C,2025-03-27,...,99098.730,98450.100,NORTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26783,2025-05-06,SN,YouTube,GreenFarm,GreenFarmCorePromotionMar-Apr,VRC,CPM,F25-44,35C,2025-03-27,...,11265.742,10907.211,NORTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26784,2025-05-06,SN,YouTube,GreenFarm,GreenFarmCorePromotionMar-Apr,VRC,CPM,F25-44,35C,2025-03-27,...,95780.800,95162.500,NORTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
def mapping_naming(df):
    mapping_creative_type = {
        "REA": "Video",
        "VID": "Video",
        "IMAGE": "Image",
        "BAN": "Banner",
        "AUD": "Audio",
        "TEXT": "Text",
        "DISC": "Discovery Ad",
        "RESAD": "Responsive Ad",
        "ST": "Story",
        "GIF": "Gif",
        "CARO": "Carousel",
        "REEL": "Reel",
        "MULT": "Multiple",
        "DNM": "dynamic",
        "CLT": "collec",
        "TVC": "tvc",
        "SMS": "SMS",
        "LIVESTREAM": "livetream"
    }
    mapping_campaign_type = {
        "VRC": "View Reach Campaign",
        "VVC": "Video View Campaign",
        "PMAX": "PMAX",
        "DG": "DemandGen",
        "SEM": "SEM",
        "VAC": "Video Action Campaign",
        "REA": "Reach",
        "ENG": "Engage",
        "TRF": "Traffic",
        "LEAD": "Lead",
        "THRU": "Thruplay",
        "PDIS": "Programatic Display",
        "PVID": "Programatic Video",
        "OTT": "Programatic OTT",
        "MH": "Masthead",
        "App": "App",
        "GDN": "GDN",
        "IS": "InStream",
        "BUMP": "Bumper",
        "TVN": "Trueview Nonskip",
        "LSA": "Livestream Ads",
        "CT": "Community Interaction",
        "PSA": "Product Shopping Ads",
        "VSA": "Video Shopping Ads"
    }
    mapping_platform = {
    "DV360": "DV360",
    "DV360-DISP": "DV360 Display",
    "DV360-VID": "DV360 Video (non-YouTube)",
    "DV360-YT": "DV360 YouTube",
    "GG": "Google Ads",
    "GA-YT": "Google Ads YouTube",
    "GDN": "Google Display Network",
    "D-YT": "YouTube Direct",
    "FB": "Facebook",
    "TT": "TikTok",
    "IG": "Instagram"
    }
    mapping_campaign_funnel = {
    "AW": "Awareness",
    "CSD": "Consideration",
    "CVR": "Conversion"
    }

    # Convert creative_type first
    df["Creative_Type"] = df["Creative_Type"].map(mapping_creative_type)
    df["Funnel"] = df["Funnel"].map(mapping_campaign_funnel)
    df["Format"] = df["Format"].map(mapping_campaign_type)


    # Create ad_format_plan column

mapping_naming(merged_df)

In [139]:
merged_df['Audience'] = merged_df['freetext_2'].astype(str) + '\n' + merged_df['TA'].astype(str)

merged_df

,report_date,Category,Platform,Brand,Campaign,Format,Buying_Method,TA,Region,Start_Date,...,Video_Plays_100,Content_Angle,Start_Date_total,sessions,engaged_sessions,add_to_carts,purchase_revenue,ecommerce_purchases,screen_page_views,Audience
0,2024-12-29,SBNL,TikTok,SurePreventGold,SBGPromo,Reach,CPM,F25-44,20C,2025-04-21,...,0.000,HTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Caregivers\nF25-44
1,2024-12-29,SBNL,TikTok,SurePreventGold,SBGPromo,Reach,CPM,F25-44,20C,2025-04-21,...,0.000,CTKD chung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Caregivers\nF25-44
2,2024-12-29,SBNL,TikTok,SurePreventGold,SBGPromo,Reach,CPM,F25-44,20C,2025-04-21,...,0.000,MayHutBui,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Caregivers\nF25-44
3,2024-12-29,SBNL,TikTok,SurePreventGold,SBGPromo,Reach,CPM,F25-44,20C,2025-04-21,...,0.000,Thematic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Caregivers\nF25-44
4,2024-12-29,SBNL,TikTok,SurePreventGold,SBGPromo,Reach,CPM,F25-44,20C,2025-04-21,...,0.000,Thematic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Caregivers\nF25-44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26781,2025-05-06,SN,YouTube,GreenFarm,GreenFarmCorePromotionMar-Apr,View Reach Campaign,CPM,F25-44,35C,2025-03-27,...,4945.542,NORTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mom - VN83-PR0007122500094\nF25-44
26782,2025-05-06,SN,YouTube,GreenFarm,GreenFarmCorePromotionMar-Apr,View Reach Campaign,CPM,F25-44,35C,2025-03-27,...,98450.100,NORTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mom - VN83-PR0007122500094\nF25-44
26783,2025-05-06,SN,YouTube,GreenFarm,GreenFarmCorePromotionMar-Apr,View Reach Campaign,CPM,F25-44,35C,2025-03-27,...,10907.211,NORTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mom - VN83-PR0007122500094\nF25-44
26784,2025-05-06,SN,YouTube,GreenFarm,GreenFarmCorePromotionMar-Apr,View Reach Campaign,CPM,F25-44,35C,2025-03-27,...,95162.500,NORTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mom - VN83-PR0007122500094\nF25-44


In [140]:
merged_df['Start_Date'] = merged_df['Start_Date'].replace(pd.to_datetime('2025-04-03'), pd.to_datetime('2025-03-07'))
merged_df['Campaign_code'] = merged_df['Campaign_code'].str.replace('-0304-0405', '-0703-0405')
merged_df['Campaign_code'] = merged_df['Campaign_code'].str.replace('Optimum Gold', 'Optimum')
merged_df['Campaign_code'] = merged_df['Campaign_code'].str.replace('Optimum Colos', 'Optimum')


In [143]:
from sqlalchemy import create_engine
import pandas as pd

DB_TYPE = 'mysql'  # Change to 'postgresql' for PostgreSQL
DB_HOST = '10.0.0.3'
DB_PORT = '3306'  # Change for different databases
DB_USER = 'dashboard_report_vinamilk'
DB_PASS = '6vtRhNxa79zPsfs5'
DB_NAME = 'dashboard_report_vinamilk'

try:
    conn = mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASS,
        database=DB_NAME,
        charset="utf8mb4"
    )
    
    if conn.is_connected():
        cursor = conn.cursor()
        print("Connected to MySQL successfully!")
    else:
        print("Failed to connect.")

except mysql.connector.Error as e:
    print(f"Error: {e}")

# Create SQLAlchemy engine
cursor = conn.cursor()
cursor.execute("SET NAMES utf8mb4;") 
cursor.execute("SET CHARACTER SET utf8mb4;")
engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

def upload_to_mysql(df, table_name, engine, if_exists='replace'):
    """
    Uploads a DataFrame to a MySQL table.
    
    :param df: Pandas DataFrame to upload
    :param table_name: Name of the table in the database
    :param engine: SQLAlchemy engine object
    :param if_exists: What to do if the table already exists ('fail', 'replace', 'append')
    """
    try:
        # Upload DataFrame to MySQL
        df.to_sql(table_name, engine, if_exists=if_exists, index=False)
        
        print(f"Data successfully uploaded to {table_name}!")
    except Exception as e:
        print(f"Error uploading data to {table_name}: {e}")

# Example usage:
# engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
upload_to_mysql(merged_df, 'report_campaign_creative', engine)

Connected to MySQL successfully!
Data successfully uploaded to report_campaign_creative!


In [141]:
merged_df

,report_date,Category,Platform,Brand,Campaign,Format,Buying_Method,TA,Region,Start_Date,...,Video_Plays_100,Content_Angle,Start_Date_total,sessions,engaged_sessions,add_to_carts,purchase_revenue,ecommerce_purchases,screen_page_views,Audience
0,2024-12-29,SBNL,TikTok,SurePreventGold,SBGPromo,Reach,CPM,F25-44,20C,2025-04-21,...,0.000,HTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Caregivers\nF25-44
1,2024-12-29,SBNL,TikTok,SurePreventGold,SBGPromo,Reach,CPM,F25-44,20C,2025-04-21,...,0.000,CTKD chung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Caregivers\nF25-44
2,2024-12-29,SBNL,TikTok,SurePreventGold,SBGPromo,Reach,CPM,F25-44,20C,2025-04-21,...,0.000,MayHutBui,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Caregivers\nF25-44
3,2024-12-29,SBNL,TikTok,SurePreventGold,SBGPromo,Reach,CPM,F25-44,20C,2025-04-21,...,0.000,Thematic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Caregivers\nF25-44
4,2024-12-29,SBNL,TikTok,SurePreventGold,SBGPromo,Reach,CPM,F25-44,20C,2025-04-21,...,0.000,Thematic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Caregivers\nF25-44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26781,2025-05-06,SN,YouTube,GreenFarm,GreenFarmCorePromotionMar-Apr,View Reach Campaign,CPM,F25-44,35C,2025-03-27,...,4945.542,NORTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mom - VN83-PR0007122500094\nF25-44
26782,2025-05-06,SN,YouTube,GreenFarm,GreenFarmCorePromotionMar-Apr,View Reach Campaign,CPM,F25-44,35C,2025-03-27,...,98450.100,NORTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mom - VN83-PR0007122500094\nF25-44
26783,2025-05-06,SN,YouTube,GreenFarm,GreenFarmCorePromotionMar-Apr,View Reach Campaign,CPM,F25-44,35C,2025-03-27,...,10907.211,NORTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mom - VN83-PR0007122500094\nF25-44
26784,2025-05-06,SN,YouTube,GreenFarm,GreenFarmCorePromotionMar-Apr,View Reach Campaign,CPM,F25-44,35C,2025-03-27,...,95162.500,NORTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mom - VN83-PR0007122500094\nF25-44


In [142]:
merged_df[merged_df['Campaign'] == 'PackageChange'].Content_Angle.unique()

array(['Puirty Award', 'VID', 'VIDPost1', 'VIDPost10', 'VIDPost11',
       'VIDPost12', 'VIDPost2', 'VIDPost3', 'VIDPost4', 'VIDPost5',
       'VIDPost6', 'VIDPost7', 'VIDPost8', 'VIDPost9', 'Package Change',
       'TGD', 'Promotion', 'HCP', 'Award'], dtype=object)